In [ ]:
import os

os.chdir(os.pardir)

%reload_ext autoreload
%autoreload 2

In [ ]:
from src.utils import factory
from src.utils.config import process_config

from src.dataloaders.data_loader import DataLoader
from src.trainers.trainer import Trainer
from src.evaluators.evaluator import Evaluator

# Specify Config File Here

Specify the name of the config file from the `configs` folder,

```
└───src
    ├───configs
    │   ├───dpn92.json
    │   ├───inceptionv4.json
    │   ├───se_resnext101.json
    │   ├───...
```

In [ ]:
CONFIG_FILE = "se_resnext101.json"

# 1. Hyperparameters

In [ ]:
CONFIG_PATH = os.path.join('src', 'configs', CONFIG_FILE)
config = process_config(CONFIG_PATH)

In [ ]:
EXP_NAME = config['exp']['name']

BATCH_SIZE = config['trainer']['batch_size']

LR_1 = config['trainer']['lr_stage_1']
N_EPOCHS_1 = config['trainer']['n_epochs_stage_1']

# Discriminative layer training
# Learning rate increases from the lower layers to the upper layers
LR_2 = slice(*config['trainer']['lr_stage_2'])
N_EPOCHS_2 = config['trainer']['n_epochs_stage_2']

TTA = config['evaluator']['tta']

# 2. Load Model

In [ ]:
model_name = config['model']['name']
model = factory.create("src.models." + model_name)()

In [ ]:
model_config = model.get_model_config()
img_stats = model.get_img_stats()
img_size = model.get_img_size()

# 3. Load Data

In [ ]:
data_loader = DataLoader()
data_bunch = data_loader.get_data_bunch(img_size=img_size, img_stats=img_stats, batch_size=BATCH_SIZE)

In [ ]:
data_bunch.show_batch(rows=3, figsize=(11,11))

# 4. Begin Training

In [ ]:
trainer = Trainer(
    data_bunch=data_bunch, 
    mixup=True, 
    exp_name=EXP_NAME,
    **model_config)

## 4.1 Stage-1: Transfer Learning

Train the classifier head first.

Only the model weights that yields the lowest validation loss in this training run will be saved and loaded at the end of training.

In [ ]:
trainer.lr_find(freeze=True)

In [ ]:
trainer.train(n_epochs=N_EPOCHS_1, max_lr=LR_1, freeze=True, name='stage_1')

In [ ]:
trainer.plot_losses()

## 4.2 Stage-2: Fine-tuning

Unfreeze all layers and train with discriminative layer training.

Only the model weights that yields the lowest validation loss in this training run will be saved and loaded at the end of training.

In [ ]:
trainer.lr_find(freeze=False)

In [ ]:
trainer.train(n_epochs=N_EPOCHS_2, max_lr=LR_2, freeze=False, name='stage_2')

In [ ]:
trainer.plot_losses()

# 5. Export Learner/Model

Model will be saved in the `saved` folder.

In [ ]:
trainer.export()

# 6. Evaluation

In [ ]:
evaluator = Evaluator(
    learn_name=EXP_NAME,
    tta=TTA,
    exp_name=EXP_NAME
)

In [ ]:
evaluator.get_accuracy()